# What we are going to do here is get model scores for Method of Victory models (mov)

In [1]:
import pandas as pd
import numpy as np
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegressionCV
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from sklearn.mixture import DPGMM

In [2]:
import sys
sys.path.append('../../automated_model_creation') #We need to access the function file

In [3]:
from functions import *


In [4]:
is_prod = False

if is_prod:
    model_file = '../../data/production_models_mov.csv'
else: #If not production then test
    model_file = '../../data/models_mov.csv'

In [5]:
#Load models
with open(model_file, newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models[0]))


19


In [6]:
model_file

'../../data/models_mov.csv'

In [7]:
df = pd.read_csv("../../data/kaggle_data/ufc-master.csv")

In [8]:
#Load the matches that have already occurred 

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]

#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [9]:
def return_finish_type(winner, finish):
    #print(winner, finish)
    #Why overcomplicate things?  We can just use a few if statements
    if winner == 'Red':
        #print("HI")
        if finish in ['U-DEC', 'S-DEC', 'M-DEC']:
            return ('Red - DEC')
        if finish in ['SUB']:
            return('Red - SUB')
        if finish in ['KO/TKO', 'DQ']:
            return('Red - KO/TKO')
    if winner == 'Blue':
        if finish in ['U-DEC', 'S-DEC', 'M-DEC']:
            return ('Blue - DEC')
        if finish in ['SUB']:
            return('Blue - SUB')
        if finish in ['KO/TKO', 'DQ']:
            return('Blue - KO/TKO')
        
    #Test for NaN
    if finish != finish:
        return('')
    
    if finish == 'Overturned':
        return('')
    
    
    return ('error')

In [10]:
len(df)

4771

In [11]:
#This calls for the power of lambda!
df['finish_type'] = df.apply(lambda x: return_finish_type(x['Winner'], x['finish']), axis=1)

In [12]:
mask = df['finish_type'] == 'error'

display(df[['Winner', 'finish', 'finish_type']][mask])

,Winner,finish,finish_type


In [13]:
from sklearn import preprocessing #Used for LabelEncoder

le = preprocessing.LabelEncoder()

le.fit(df['finish_type'])

display(le.classes_)

df['label'] = le.transform(df['finish_type'])

display(df)


array(['', 'Blue - DEC', 'Blue - KO/TKO', 'Blue - SUB', 'Red - DEC',
       'Red - KO/TKO', 'Red - SUB'], dtype=object)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,R_rev_bout,B_rev_bout,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,finish_type,label
0,Ciryl Gane,Alexander Volkov,-155,125,64.516129,125.0,2021-06-26,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,188.0,300.0,700.0,375.0,275.0,1600.0,Red - DEC,4
1,Tanner Boser,Ovince Saint Preux,-195,153,51.282051,153.0,2021-06-26,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,175.0,250.0,1800.0,350.0,700.0,500.0,Red - KO/TKO,5
2,Raoni Barcelos,Timur Valiev,-230,180,43.478261,180.0,2021-06-26,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,125.0,375.0,600.0,325.0,800.0,1250.0,Blue - DEC,1
3,Tim Means,Nicolas Dalby,-143,113,69.930070,113.0,2021-06-26,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,165.0,400.0,800.0,225.0,600.0,1000.0,Red - DEC,4
4,Renato Moicano,Jai Herbert,-250,195,40.000000,195.0,2021-06-26,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,240.0,550.0,175.0,550.0,325.0,1500.0,Red - SUB,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4766,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.0,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Blue - KO/TKO,2
4767,John Howard,Daniel Roberts,-210,175,47.619048,175.0,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,Red - KO/TKO,5
4768,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.0,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Red - KO/TKO,5
4769,Mike Pierce,Julio Paulino,-420,335,23.809524,335.0,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Red - DEC,4


In [14]:
label_df = df['label']
df_odds = df[['r_dec_odds', 'r_sub_odds', 'r_ko_odds', 'b_dec_odds', 'b_sub_odds', 'b_ko_odds']]


df_train = df[250:]
label_train = label_df[250:]
odds_train = df_odds[250:]

df_test = df[:250]
label_test = label_df[:250]
odds_test = df_odds[:250]

print(len(df_test))
print(len(label_test))
print(len(odds_test))


250
250
250


In [15]:
weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)

C:\Users\matth\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [16]:
#We need to split then mask.  The models can never see the first 250 rows.

mask = df_train['finish_type'] != ''
df_train = df_train[mask]
#print(len(df_train))

mask = df_test['finish_type'] != ''
df_test = df_test[mask]
#print(len(df_test))

label_train = label_train[label_train.index.isin(df_train.index)]
label_test = label_test[label_test.index.isin(df_test.index)]

odds_train = odds_train[odds_train.index.isin(df_train.index)]
odds_test = odds_test[odds_test.index.isin(df_test.index)]


print(len(df_train))
print(len(label_train))
print(len(odds_train))
print(len(df_test))
print(len(label_test))
print(len(odds_test))

4116
4116
4116
50
50
50


In [17]:
odds_train = odds_train.dropna()
odds_test = odds_test.dropna()

df_train = df_train[df_train.index.isin(odds_train.index)]
df_test = df_test[df_test.index.isin(odds_test.index)]

label_train = label_train[label_train.index.isin(odds_train.index)]
label_test = label_test[label_test.index.isin(odds_test.index)]

print(len(df_train))
print(len(label_train))
print(len(odds_train))
print(len(df_test))
print(len(label_test))
print(len(odds_test))

3082
3082
3082
49
49
49


In [18]:
model_num = 0

In [19]:
print(odds_test.columns)
print(le.classes_)

Index(['r_dec_odds', 'r_sub_odds', 'r_ko_odds', 'b_dec_odds', 'b_sub_odds',
       'b_ko_odds'],
      dtype='object')
['' 'Blue - DEC' 'Blue - KO/TKO' 'Blue - SUB' 'Red - DEC' 'Red - KO/TKO'
 'Red - SUB']


In [20]:
odds_test.rename(columns={'r_dec_odds': 'Red - DEC', 'r_sub_odds': 'Red - SUB', 'r_ko_odds': 'Red - KO/TKO',
                'b_dec_odds': 'Blue - DEC', 'b_sub_odds': 'Blue - SUB', 'b_ko_odds': 'Blue - KO/TKO'}, inplace=True)
odds_train.rename(columns={'r_dec_odds': 'Red - DEC', 'r_sub_odds': 'Red - SUB', 'r_ko_odds': 'Red - KO/TKO',
                'b_dec_odds': 'Blue - DEC', 'b_sub_odds': 'Blue - SUB', 'b_ko_odds': 'Blue - KO/TKO'}, inplace=True)


In [21]:
score_list = []

In [22]:
score_list = []
for model_num in range(len(models[0])):
    test_model_name = models[0][model_num]
    test_model = eval(models[1][model_num])
    test_model_features = eval(models[2][model_num])
    test_model_ev = eval(models[3][model_num])
    score_list.append(evaluate_model_mov(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, le.classes_, verbose = True))



(3082, 20)

(3082,)

(3082, 6)

(49, 20)

(49,)

...

(49, 17)

(49,)

(49, 6)




   Red - DEC  Red - SUB  Red - KO/TKO  Blue - DEC  Blue - SUB  Blue - KO/TKO
0      188.0      700.0         275.0       300.0       375.0         1600.0
Blue - DEC: 0.139501332254642
300.0
-44.199467098143195

Blue - KO/TKO: 0.16084239978994658
1600.0
173.43207964290917
1
Losing Bet.  New Score: -100

Blue - SUB: 0.05594373188727306
375.0
-73.4267273535453

Red - DEC: 0.2090853967112852
188.0
-39.78340574714986

Red - KO/TKO: 0.3519353639859842
275.0
31.975761494744063
4
Losing Bet.  New Score: -200

Red - SUB: 0.08269177537086878
700.0
-33.84657970330498

Result: Red - DEC (4)
   Red - DEC  Red - SUB  Red - KO/TKO  Blue - DEC  Blue - SUB  Blue - KO/TKO
1      175.0     1800.0         700.0       250.0       350.0          500.0
Blue - DEC: 0.20143535353012448
250.0
-29.49762626445643

Blue - KO/TKO: 0.18549494392066113
500.0
11.296966352396666
1
Losing Bet.  New Score: -300

Blue - SUB: 0.07626300232916412
350.0
-65.68164895187613

Red - DEC: 0.1839905104160161
175.0
-49.402609635

1800.0
59.25791955723108
2
Losing Bet.  New Score: 1675.0

Red - DEC: 0.3506093954243056
650.0
162.95704656822917
3
Losing Bet.  New Score: 1575.0

Red - KO/TKO: 0.15321480887617295
1100.0
83.85777065140753
4
5 5
Winning Bet. New Score: 2675.0

Red - SUB: 0.10769060596534107
425.0
-43.46243186819594

Result: Red - KO/TKO (5)
    Red - DEC  Red - SUB  Red - KO/TKO  Blue - DEC  Blue - SUB  Blue - KO/TKO
28      185.0     1200.0         300.0       250.0       800.0          500.0
Blue - DEC: 0.3087175632547069
250.0
8.051147139147403
0
1 1
Winning Bet. New Score: 2925.0

Blue - KO/TKO: 0.157466677433179
500.0
-5.5199935400925995

Blue - SUB: 0.07725587450300349
800.0
-30.469712947296856

Red - DEC: 0.2456814059397743
185.0
-29.980799307164318

Red - KO/TKO: 0.12954785179834838
300.0
-48.18085928066064

Red - SUB: 0.0813306270709876
1200.0
5.729815192283866
5
Losing Bet.  New Score: 2825.0

Result: Blue - DEC (1)
    Red - DEC  Red - SUB  Red - KO/TKO  Blue - DEC  Blue - SUB  Blue - KO/TK

In [23]:
models[4] = score_list

In [24]:
with open(model_file, 'w' , newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        #print("HI")
        writer.writerow(row)
    
outfile.close()

In [25]:
print(score_list)

[1510.0, 3158.0, 1163.0, 1459.9565217391305, -640.0, 1550.0, 2075.0, -447.92592592592587, 679.9565217391305, -1775.0, 1588.0, -1125.0, 3.0, 1798.0, 2055.0, 695.0, 550.0, -40.0, 1710.0]


In [26]:
models[4] = score_list

In [27]:
print(score_list)

[1510.0, 3158.0, 1163.0, 1459.9565217391305, -640.0, 1550.0, 2075.0, -447.92592592592587, 679.9565217391305, -1775.0, 1588.0, -1125.0, 3.0, 1798.0, 2055.0, 695.0, 550.0, -40.0, 1710.0]
